In [1]:
import pyomo.environ as pe
import pandas as pd
import numpy as np
import pdb

pd.options.display.max_colwidth = 100

In [2]:
class AvgCostMDP:
    def __init__(self,InDir,filename,actions,alpha=0.9):#,absorbing_states=[]):
        self.InDir=InDir
        self.filename=filename
        self.actions=actions #these will be the sheetnames in above file that we want to read
        self.states=[]
        self.states_actions={}
        self.transitions={}
        self.qalys={}
        self.populate_states() #tpm: transition probability matrix
        self.populate_state_dependent_actions()
        self.qaly_func()
        self.alpha=alpha
#         self.transient_states=[s for s in self.states if s not in absorbing_states]
        
    def populate_states(self):
        for a in self.actions:
            try:
                df=pd.read_excel(self.InDir+'\\'+self.filename,sheetname=a)
            except:
                print('\n\n!!!...Pandas read error...!!!\n\n')
                pdb.set_trace()
            try:
                df=df.set_index(df.columns[0])
                self.transitions[a]=df
            except:
                print('\n\n!!!...Pandas set index key error...!!!\n\n')
                pdb.set_trace()
            try:
                self.states=np.append(self.states,[s for s in list(df.index) if s not in self.states])
                self.states=np.append(self.states,[s for s in list(df.columns) if s not in self.states])
            except:
                print('\n\n!!!...State space error...!!!\n\n')
                pdb.set_trace()
        
    def populate_state_dependent_actions(self):
        for s in self.states:
            self.states_actions[s]=[]
        for a in self.transitions:
            transition_df=self.transitions[a]
            for s in list(transition_df.index):
#                 if s=='Seizure free with drugs after Workup to Surgery with 3rd AED':
#                     pdb.set_trace()
                if a not in self.states_actions[s]:
                    self.states_actions[s]=np.append(self.states_actions[s],a)# if a not in self.states_actions[s] else continue
    
    def qaly_func(self):
        if qaly=='readFile':
            df=pd.read_excel(self.InDir+'\\'+self.filename,sheetname="QALY for state")
            df=df[['State','Reward']] #We want to drop any other columns in the data file such
            for s in self.states:
                self.qalys[s]={}
                for a in self.actions:
#                     try:
#                         if a in self.states_actions[s]:
#                             q=-float(df[df['State']==s]['Reward'])
#                     except:
#                         pdb.set_trace()
                    self.qalys[s][a]=float(df[df['State']==s]['Reward']) if a in self.states_actions[s] else 0.0                  
        elif qaly==1:
            for s in self.states:
                self.qalys[s]={}
                for a in self.actions:
                    self.qalys[s][a]=1.0 if a in self.states_actions[s] else 0.0        

## Data Log
#### Here I have logged the major differences between the data files

*Note: Everytime you make any changes to the transition probability of any action in the sheet "Simpler MDP" in the data spreadsheet, make sure you make the changes in the corresponding action's sheet in the same spreadsheet too. Then re-read the new spreadsheet below and run all the cells again.*

- **Data_required_2:**
    - Changed generic state names to action specific state names. E.g., *No Improvement* was modified to *No Improvement w/o AED*, *No Improvement with 1st AED*, *No Improvement with 2nd AED*, and so on.
    - Also changed first column of *Workup to Surgery with 3rd AED* to be *No Improvement with 3rd AED* in stead of *No Improvement with 3rd AED*. The reasoning being that no change is caused due to the workup since it's not a treatment but only a procedure to determine suitability for surgery. The treatment here is actually the 3rd AED.
- **Data_required_3:**
    - Added rows for *Refractory Epilepsy* and *Adverse Outcomes* for actions *Surgery or Resection*, * AED after Seizure freedom* and *Discontinue AED*. This helped represent these absorbing states under these actions.
    - Deleted the column *Improved but not controlled by 3rd AED* and *Improved but not controlled by Workup to Surgery with 3rd AED* for the action transition probability matrices correspondong *3rd AED* and *Workup to Surgery with 3rd AED*, respectively becasue no state was leading to a transition into these states under the two actions. **Ask Dr. Clarke about this and get the data or data source if these transitions do exist.** 
    - Changed first column of *Workup to Surgery with 3rd AED* back to *No Improvement with 3rd AED* from *No Improvement with 3rd AED* for convenience. **I am leaning towards reverting this again and also reducing the corresponding four rows under action *Surgery or resection to two rows* because it logically makes sense since *Workup to Surgery* is not a treatmdnt.**
    - Added all the *Seizrue free after 1st AED*, *Seizrue free after 2nd AED*, etc., rows to the last action *Discontinue AED*.
- **Data_required_4:** Changed row state names of first action *1st AED* from *No Improvement*, *Poor control*, *Improved but not controlled* and *Refractory Epilepsy* to *No Improvement w/o AED*, *Poor control w/o AED*, *Improved but not controlled w/o AED* and *Refractory Epilepsy w/o AED* to *Refractory Epilepsy w/o AED*, respectively.
- **Data_required_5:**
    - Removed *Adverse Outcomes* as a row from each action matrix (compare with *Data_required_4*)
    - Changed row state name of first action *1st AED* from *Refractory Epilepsy w/o AED* to *Refractory Epilepsy* because refractory epilepsy is refractory epilepsy and should be independent of the treatment being used. *CONFIRM this with Dr. Clarke".
- **Data_required_6:**
    - Added rows "No Improvement by x action", "Poor control by x action" and "Improved but not controlled by x action" as recurrent states to actions "1st AED", "2nd AED" and "Workup to Surgery"; and "No Improvement by x action" and "Poor control by x action" as recurrent states to actions "3rd AED" and "Workup to Surgery with 3rd AED".

In [3]:
# %%time
InDir=r"C:\Users\Shreya\Dropbox\Shreya\Research Shreya\Epilepsy\Model"
filename='Data_Required_12_.xlsx'    
actions=['1st AED',
         '2nd AED',
#          '3rd AED',
#          'Workup to Surgery',
         'Workup to Surgery with 3rd AED',
         'Surgery or resection',
         'Medical Management',
         'AED after SF',
         'Discontinue AED']   
# absorbing_states=['Refractory Epilepsy','Adverse Outcomes','']
qaly='readFile'
mdp=AvgCostMDP(InDir,filename,actions,qaly)

num_states=len(mdp.states)
num_actions=len(mdp.actions)

ai_s_RHS=[np.round(1.0/num_states,3)]*num_states
ai_s_RHS[0]=0.04#1-sum(ai_s_RHS[1:])

#------ Model Definition -------

# del model
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
model.states  = pe.Set(initialize = range(num_states))
# model.states  = pe.Set(initialize = range(num_states))
model.actions  = pe.Set(initialize = range(num_actions))
def cost_func(model,state,action): 
    return mdp.qalys[mdp.states[state]][mdp.actions[action]]
model.cost = pe.Param(model.states, model.actions, initialize = cost_func)
model.ai_s = pe.Param(model.states,   initialize = lambda model,s: ai_s_RHS[s]) #'s' is the index num of corresponding state
model.X = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)
model.Y = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)

#------------- CONSTRAINTS -----------
                         
def avg_cost_mdp_dual_constraint1(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_probab_transitions_with_X=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
#             print(j,a)
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
#                         if j==23:
#                             pdb.set_trace()
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_X=sum_over_probab_transitions_with_X + pij*model.X[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X - sum_over_probab_transitions_with_X == 0)
model.Constraint1 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint1)
model.Constraint1.pprint()

def avg_cost_mdp_dual_constraint2(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_actions_Y = sum(model.Y[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]]) 
    sum_over_probab_transitions_with_Y=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
#             print(j,a)
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
#                         if j==23:
#                             pdb.set_trace()
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_Y=sum_over_probab_transitions_with_Y + pij*model.Y[i,a]  
                    except:
                        pij=0.0
#     model.ai_s.pprint()
#     print('constraint for state '+str(i))
#     if i==15:
#         print(mdp.states[i])
#         pdb.set_trace()
    return(sum_over_actions_X + sum_over_actions_Y - sum_over_probab_transitions_with_Y == model.ai_s[j])
model.Constraint2 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint2)
model.Constraint2.pprint()

#------ Objective Function ------

def obj_rule(model):
    return sum(model.cost[s,a] * model.X[s,a] for s in model.states for a in model.actions)
model.OBJ = pe.Objective(rule = obj_rule, sense = pe.maximize)
model.OBJ.pprint()

#------ Solve LP ------

solver = pe.SolverFactory('gurobi') # Specify Solver
results = solver.solve(model, tee = False, keepfiles = False)
print("\n\nStatus:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# --------- POST-PROCESSING -------------------

states=[]
actions=[]
dual_vals=[]
for s in model.states:
    for a in model.actions:
        try:
            if (model.X[s,a].value != 0 and model.X[s,a].value != None) or (model.Y[s,a].value != 0 and model.Y[s,a].value != None):
                states=np.append(states,mdp.states[s])
                actions=np.append(actions,mdp.actions[a])
                dual_vals=np.append(dual_vals,np.round(model.X[s,a].value,10))
        except:
            print('Suspicious value at state %d action %d'%(s,a),model.X[s,a].value)
            continue
df=pd.DataFrame.from_dict(dict(state=states,action=actions,dual_val=dual_vals))
print("\n\nObjective function value: ", model.OBJ())
df[['state','action','dual_val']]

Constraint1 : Size=49, Index=states, Active=True
    Key : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

,state,action,dual_val
0,No Improvement w/o AED,1st AED,0.000000
1,Poor control w/o AED,1st AED,0.000000
2,Improved but not controlled w/o AED,1st AED,0.000000
3,Refractory epilepsy,1st AED,0.106724
4,No Improvement with 1st AED,2nd AED,0.000000
5,Poor control with 1st AED,2nd AED,0.000000
6,Improved but not controlled with 1st AED,1st AED,0.000000
7,Seizure free with drugs after 1st AED,Discontinue AED,0.000000
8,Relapse after SF when AED stopped,1st AED,0.000000
9,Poor control w/o AED after Relapse,1st AED,0.000000


In [5]:
df.to_excel("output_textbook_solution.xlsx")

# Surgery:
 - Adverse outcome 22.48% (0.29)
 - Even a probab as low as 1% for Completely SF does not change probab
 - Even a probab as high as 99% for No Change does not change probab

In [5]:
# %%time
InDir=r"C:\Users\Shreya\Dropbox\Shreya\Research Shreya\Epilepsy\Model"
filename='Data_Required_12b.xlsx'    
actions=['1st AED',
         '2nd AED',
#          '3rd AED',
#          'Workup to Surgery',
         'Workup to Surgery with 3rd AED',
         'Surgery or resection',
         'Medical Management',
         'AED after SF',
         'Discontinue AED']   
# absorbing_states=['Refractory Epilepsy','Adverse Outcomes','']
qaly='readFile'
mdp=AvgCostMDP(InDir,filename,actions,qaly)

num_states=len(mdp.states)
num_actions=len(mdp.actions)

ai_s_RHS=[np.round(1.0/num_states,3)]*num_states
ai_s_RHS[0]=0.04#1-sum(ai_s_RHS[1:])

#------ Model Definition -------

# del model
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
model.states  = pe.Set(initialize = range(num_states))
# model.states  = pe.Set(initialize = range(num_states))
model.actions  = pe.Set(initialize = range(num_actions))
def cost_func(model,state,action): 
    return mdp.qalys[mdp.states[state]][mdp.actions[action]]
model.cost = pe.Param(model.states, model.actions, initialize = cost_func)
model.ai_s = pe.Param(model.states,   initialize = lambda model,s: ai_s_RHS[s]) #'s' is the index num of corresponding state
model.X = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)
model.Y = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)

#------------- CONSTRAINTS -----------
                         
def avg_cost_mdp_dual_constraint1(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_probab_transitions_with_X=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_X=sum_over_probab_transitions_with_X + pij*model.X[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X - sum_over_probab_transitions_with_X == 0)
model.Constraint1 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint1)
# model.Constraint1.pprint()

def avg_cost_mdp_dual_constraint2(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_actions_Y = sum(model.Y[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]]) 
    sum_over_probab_transitions_with_Y=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_Y=sum_over_probab_transitions_with_Y + pij*model.Y[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X + sum_over_actions_Y - sum_over_probab_transitions_with_Y == model.ai_s[j])
model.Constraint2 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint2)
# model.Constraint2.pprint()

#------ Objective Function ------

def obj_rule(model):
    return sum(model.cost[s,a] * model.X[s,a] for s in model.states for a in model.actions)
model.OBJ = pe.Objective(rule = obj_rule, sense = pe.maximize)
# model.OBJ.pprint()

#------ Solve LP ------

solver = pe.SolverFactory('gurobi') # Specify Solver
results = solver.solve(model, tee = False, keepfiles = False)
print("\n\nStatus:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# --------- POST-PROCESSING -------------------

states=[]
actions=[]
dual_vals=[]
for s in model.states:
    for a in model.actions:
        try:
            if (model.X[s,a].value != 0 and model.X[s,a].value != None) or (model.Y[s,a].value != 0 and model.Y[s,a].value != None):
                states=np.append(states,mdp.states[s])
                actions=np.append(actions,mdp.actions[a])
                dual_vals=np.append(dual_vals,np.round(model.X[s,a].value,10))
        except:
            print('Suspicious value at state %d action %d'%(s,a),model.X[s,a].value)
            continue
df=pd.DataFrame.from_dict(dict(state=states,action=actions,dual_val=dual_vals))
print("\n\nObjective function value: ", model.OBJ())
df[['state','action','dual_val']]



Status: ok
Termination Condition: optimal


Objective function value:  543.7394049679001


,state,action,dual_val
0,No Improvement w/o AED,1st AED,0.000000
1,Poor control w/o AED,1st AED,0.000000
2,Improved but not controlled w/o AED,1st AED,0.000000
3,Refractory epilepsy,2nd AED,0.000000
4,No Improvement with 1st AED,2nd AED,0.000000
5,Poor control with 1st AED,2nd AED,0.000000
6,Improved but not controlled with 1st AED,2nd AED,0.000000
7,Seizure free with drugs after 1st AED,Discontinue AED,0.000000
8,Relapse after SF when AED stopped,1st AED,0.000000
9,Poor control w/o AED after Relapse,1st AED,0.000000


# Flipped Outcomes:
 - **1st AED**: Adverse outcome >=20.11% (0.234) when in state Improved but not controlled flips recommended action from continue 1st AED to 2nd AED
 - **2nd AED, 3rd AEd and Workup to Surgery**: In almost all situations that we tried, the model recommends Workup to Surgery if the patient does not become SF with the 2nd AED. 
 - **Surgery**: Adverse outcome probability >=22.48% (0.29) flips decision from Surgery to Medical Management
 - When the QALYs are very close to each other, e.g., for a very old patient; the model recommends not changing the treatment in any state and simply continuing with whatever the current treatment is. It recommends so at a loss of QALY but clearly this loss in QALY is the maximum possible QALY and QALYs will drop further if the model is changed more. In our model we chose the QALYs to be ...
 - QALYs only with even two units of difference between each other leads to textbook solution

# QALYs only with even two units of difference between each other leads to textbook solution 

In [12]:
# %%time
InDir=r"C:\Users\Shreya\Dropbox\Shreya\Research Shreya\Epilepsy\Model"
filename='Data_Required_12_.xlsx'    
actions=['1st AED',
         '2nd AED',
         '3rd AED',
#          'Workup to Surgery',
         'Workup to Surgery with 3rd AED',
         'Surgery or resection',
         'Medical Management',
         'AED after SF',
         'Discontinue AED']   
# absorbing_states=['Refractory Epilepsy','Adverse Outcomes','']
qaly='readFile'
mdp=AvgCostMDP(InDir,filename,actions,qaly)

num_states=len(mdp.states)
num_actions=len(mdp.actions)

ai_s_RHS=[np.round(1.0/num_states,3)]*num_states
ai_s_RHS[0]=0.04#1-sum(ai_s_RHS[1:])

#------ Model Definition -------

# del model
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
model.states  = pe.Set(initialize = range(num_states))
# model.states  = pe.Set(initialize = range(num_states))
model.actions  = pe.Set(initialize = range(num_actions))
def cost_func(model,state,action): 
    return mdp.qalys[mdp.states[state]][mdp.actions[action]]
model.cost = pe.Param(model.states, model.actions, initialize = cost_func)
model.ai_s = pe.Param(model.states,   initialize = lambda model,s: ai_s_RHS[s]) #'s' is the index num of corresponding state
model.X = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)
model.Y = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)

#------------- CONSTRAINTS -----------
                         
def avg_cost_mdp_dual_constraint1(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_probab_transitions_with_X=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_X=sum_over_probab_transitions_with_X + pij*model.X[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X - sum_over_probab_transitions_with_X == 0)
model.Constraint1 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint1)
# model.Constraint1.pprint()

def avg_cost_mdp_dual_constraint2(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_actions_Y = sum(model.Y[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]]) 
    sum_over_probab_transitions_with_Y=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_Y=sum_over_probab_transitions_with_Y + pij*model.Y[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X + sum_over_actions_Y - sum_over_probab_transitions_with_Y == model.ai_s[j])
model.Constraint2 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint2)
# model.Constraint2.pprint()

#------ Objective Function ------

def obj_rule(model):
    return sum(model.cost[s,a] * model.X[s,a] for s in model.states for a in model.actions)
model.OBJ = pe.Objective(rule = obj_rule, sense = pe.maximize)
# model.OBJ.pprint()

#------ Solve LP ------

solver = pe.SolverFactory('gurobi') # Specify Solver
results = solver.solve(model, tee = False, keepfiles = False)
print("\n\nStatus:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# --------- POST-PROCESSING -------------------

states=[]
actions=[]
dual_vals=[]
for s in model.states:
    for a in model.actions:
        try:
            if (model.X[s,a].value != 0 and model.X[s,a].value != None) or (model.Y[s,a].value != 0 and model.Y[s,a].value != None):
                states=np.append(states,mdp.states[s])
                actions=np.append(actions,mdp.actions[a])
                dual_vals=np.append(dual_vals,np.round(model.X[s,a].value,10))
        except:
            print('Suspicious value at state %d action %d'%(s,a),model.X[s,a].value)
            continue
df=pd.DataFrame.from_dict(dict(state=states,action=actions,dual_val=dual_vals))
print("\n\nObjective function value: ", model.OBJ())
df[['state','action','dual_val']]



Status: ok
Termination Condition: optimal


Objective function value:  2.243533915308


,state,action,dual_val
0,No Improvement w/o AED,1st AED,0.000000
1,Poor control w/o AED,1st AED,0.000000
2,Improved but not controlled w/o AED,1st AED,0.000000
3,Refractory epilepsy,1st AED,0.115520
4,No Improvement with 1st AED,2nd AED,0.000000
5,Poor control with 1st AED,2nd AED,0.000000
6,Improved but not controlled with 1st AED,1st AED,0.000000
7,Seizure free with drugs after 1st AED,Discontinue AED,0.000000
8,Relapse after SF when AED stopped,1st AED,0.000000
9,Poor control w/o AED after Relapse,1st AED,0.000000


# Q3*4 

In [13]:
# %%time
InDir=r"C:\Users\Shreya\Dropbox\Shreya\Research Shreya\Epilepsy\Model"
filename='Data_Required_12_.xlsx'    
actions=['1st AED',
         '2nd AED',
         '3rd AED',
#          'Workup to Surgery',
         'Workup to Surgery with 3rd AED',
         'Surgery or resection',
         'Medical Management',
         'AED after SF',
         'Discontinue AED']   
# absorbing_states=['Refractory Epilepsy','Adverse Outcomes','']
qaly='readFile'
mdp=AvgCostMDP(InDir,filename,actions,qaly)

num_states=len(mdp.states)
num_actions=len(mdp.actions)

ai_s_RHS=[np.round(1.0/num_states,3)]*num_states
ai_s_RHS[0]=0.04#1-sum(ai_s_RHS[1:])

#------ Model Definition -------

# del model
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
model.states  = pe.Set(initialize = range(num_states))
# model.states  = pe.Set(initialize = range(num_states))
model.actions  = pe.Set(initialize = range(num_actions))
def cost_func(model,state,action): 
    return mdp.qalys[mdp.states[state]][mdp.actions[action]]
model.cost = pe.Param(model.states, model.actions, initialize = cost_func)
model.ai_s = pe.Param(model.states,   initialize = lambda model,s: ai_s_RHS[s]) #'s' is the index num of corresponding state
model.X = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)
model.Y = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)

#------------- CONSTRAINTS -----------
                         
def avg_cost_mdp_dual_constraint1(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_probab_transitions_with_X=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_X=sum_over_probab_transitions_with_X + pij*model.X[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X - sum_over_probab_transitions_with_X == 0)
model.Constraint1 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint1)
# model.Constraint1.pprint()

def avg_cost_mdp_dual_constraint2(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_actions_Y = sum(model.Y[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]]) 
    sum_over_probab_transitions_with_Y=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_Y=sum_over_probab_transitions_with_Y + pij*model.Y[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X + sum_over_actions_Y - sum_over_probab_transitions_with_Y == model.ai_s[j])
model.Constraint2 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint2)
# model.Constraint2.pprint()

#------ Objective Function ------

def obj_rule(model):
    return sum(model.cost[s,a] * model.X[s,a] for s in model.states for a in model.actions)
model.OBJ = pe.Objective(rule = obj_rule, sense = pe.maximize)
# model.OBJ.pprint()

#------ Solve LP ------

solver = pe.SolverFactory('gurobi') # Specify Solver
results = solver.solve(model, tee = False, keepfiles = False)
print("\n\nStatus:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# --------- POST-PROCESSING -------------------

states=[]
actions=[]
dual_vals=[]
for s in model.states:
    for a in model.actions:
        try:
            if (model.X[s,a].value != 0 and model.X[s,a].value != None) or (model.Y[s,a].value != 0 and model.Y[s,a].value != None):
                states=np.append(states,mdp.states[s])
                actions=np.append(actions,mdp.actions[a])
                dual_vals=np.append(dual_vals,np.round(model.X[s,a].value,10))
        except:
            print('Suspicious value at state %d action %d'%(s,a),model.X[s,a].value)
            continue
df=pd.DataFrame.from_dict(dict(state=states,action=actions,dual_val=dual_vals))
print("\n\nObjective function value: ", model.OBJ())
df[['state','action','dual_val']]



Status: ok
Termination Condition: optimal


Objective function value:  4.487067830616


,state,action,dual_val
0,No Improvement w/o AED,1st AED,0.000000
1,Poor control w/o AED,1st AED,0.000000
2,Improved but not controlled w/o AED,1st AED,0.000000
3,Refractory epilepsy,1st AED,0.115520
4,No Improvement with 1st AED,2nd AED,0.000000
5,Poor control with 1st AED,2nd AED,0.000000
6,Improved but not controlled with 1st AED,1st AED,0.000000
7,Seizure free with drugs after 1st AED,Discontinue AED,0.000000
8,Relapse after SF when AED stopped,1st AED,0.000000
9,Poor control w/o AED after Relapse,1st AED,0.000000


# 1st AED:Adverse Outcome at 20.11%

In [14]:
# %%time
InDir=r"C:\Users\Shreya\Dropbox\Shreya\Research Shreya\Epilepsy\Model"
filename='Data_Required_13b.xlsx'    
actions=['1st AED',
         '2nd AED',
         '3rd AED',
#          'Workup to Surgery',
         'Workup to Surgery with 3rd AED',
         'Surgery or resection',
         'Medical Management',
         'AED after SF',
         'Discontinue AED']   
# absorbing_states=['Refractory Epilepsy','Adverse Outcomes','']
qaly='readFile'
mdp=AvgCostMDP(InDir,filename,actions,qaly)

num_states=len(mdp.states)
num_actions=len(mdp.actions)

ai_s_RHS=[np.round(1.0/num_states,3)]*num_states
ai_s_RHS[0]=0.04#1-sum(ai_s_RHS[1:])

#------ Model Definition -------

# del model
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
model.states  = pe.Set(initialize = range(num_states))
# model.states  = pe.Set(initialize = range(num_states))
model.actions  = pe.Set(initialize = range(num_actions))
def cost_func(model,state,action): 
    return mdp.qalys[mdp.states[state]][mdp.actions[action]]
model.cost = pe.Param(model.states, model.actions, initialize = cost_func)
model.ai_s = pe.Param(model.states,   initialize = lambda model,s: ai_s_RHS[s]) #'s' is the index num of corresponding state
model.X = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)
model.Y = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)

#------------- CONSTRAINTS -----------
                         
def avg_cost_mdp_dual_constraint1(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_probab_transitions_with_X=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_X=sum_over_probab_transitions_with_X + pij*model.X[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X - sum_over_probab_transitions_with_X == 0)
model.Constraint1 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint1)
# model.Constraint1.pprint()

def avg_cost_mdp_dual_constraint2(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_actions_Y = sum(model.Y[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]]) 
    sum_over_probab_transitions_with_Y=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_Y=sum_over_probab_transitions_with_Y + pij*model.Y[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X + sum_over_actions_Y - sum_over_probab_transitions_with_Y == model.ai_s[j])
model.Constraint2 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint2)
# model.Constraint2.pprint()

#------ Objective Function ------

def obj_rule(model):
    return sum(model.cost[s,a] * model.X[s,a] for s in model.states for a in model.actions)
model.OBJ = pe.Objective(rule = obj_rule, sense = pe.maximize)
# model.OBJ.pprint()

#------ Solve LP ------

solver = pe.SolverFactory('gurobi') # Specify Solver
results = solver.solve(model, tee = False, keepfiles = False)
print("\n\nStatus:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# --------- POST-PROCESSING -------------------

states=[]
actions=[]
dual_vals=[]
for s in model.states:
    for a in model.actions:
        try:
            if (model.X[s,a].value != 0 and model.X[s,a].value != None) or (model.Y[s,a].value != 0 and model.Y[s,a].value != None):
                states=np.append(states,mdp.states[s])
                actions=np.append(actions,mdp.actions[a])
                dual_vals=np.append(dual_vals,np.round(model.X[s,a].value,10))
        except:
            print('Suspicious value at state %d action %d'%(s,a),model.X[s,a].value)
            continue
df=pd.DataFrame.from_dict(dict(state=states,action=actions,dual_val=dual_vals))
print("\n\nObjective function value: ", model.OBJ())
df[['state','action','dual_val']]



Status: ok
Termination Condition: optimal


Objective function value:  589.47899345471


,state,action,dual_val
0,No Improvement w/o AED,1st AED,0.000000
1,Poor control w/o AED,1st AED,0.000000
2,Improved but not controlled w/o AED,1st AED,0.000000
3,Refractory epilepsy,1st AED,0.129593
4,No Improvement with 1st AED,2nd AED,0.000000
5,Poor control with 1st AED,2nd AED,0.000000
6,Improved but not controlled with 1st AED,2nd AED,0.000000
7,Seizure free with drugs after 1st AED,Discontinue AED,0.000000
8,Relapse after SF when AED stopped,1st AED,0.000000
9,Poor control w/o AED after Relapse,1st AED,0.000000


# 1st AED Adverse 34.98% 

In [13]:
# %%time
InDir=r"C:\Users\Shreya\Dropbox\Shreya\Research Shreya\Epilepsy\Model"
filename='Data_Required_13c.xlsx'    
actions=['1st AED',
         '2nd AED',
         '3rd AED',
#          'Workup to Surgery',
         'Workup to Surgery with 3rd AED',
         'Surgery or resection',
         'Medical Management',
         'AED after SF',
         'Discontinue AED']   
# absorbing_states=['Refractory Epilepsy','Adverse Outcomes','']
qaly='readFile'
mdp=AvgCostMDP(InDir,filename,actions,qaly)

num_states=len(mdp.states)
num_actions=len(mdp.actions)

ai_s_RHS=[np.round(1.0/num_states,3)]*num_states
ai_s_RHS[0]=0.04#1-sum(ai_s_RHS[1:])

#------ Model Definition -------

# del model
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
model.states  = pe.Set(initialize = range(num_states))
# model.states  = pe.Set(initialize = range(num_states))
model.actions  = pe.Set(initialize = range(num_actions))
def cost_func(model,state,action): 
    return mdp.qalys[mdp.states[state]][mdp.actions[action]]
model.cost = pe.Param(model.states, model.actions, initialize = cost_func)
model.ai_s = pe.Param(model.states,   initialize = lambda model,s: ai_s_RHS[s]) #'s' is the index num of corresponding state
model.X = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)
model.Y = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)

#------------- CONSTRAINTS -----------
                         
def avg_cost_mdp_dual_constraint1(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_probab_transitions_with_X=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_X=sum_over_probab_transitions_with_X + pij*model.X[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X - sum_over_probab_transitions_with_X == 0)
model.Constraint1 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint1)
# model.Constraint1.pprint()

def avg_cost_mdp_dual_constraint2(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_actions_Y = sum(model.Y[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]]) 
    sum_over_probab_transitions_with_Y=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_Y=sum_over_probab_transitions_with_Y + pij*model.Y[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X + sum_over_actions_Y - sum_over_probab_transitions_with_Y == model.ai_s[j])
model.Constraint2 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint2)
# model.Constraint2.pprint()

#------ Objective Function ------

def obj_rule(model):
    return sum(model.cost[s,a] * model.X[s,a] for s in model.states for a in model.actions)
model.OBJ = pe.Objective(rule = obj_rule, sense = pe.maximize)
# model.OBJ.pprint()

#------ Solve LP ------

solver = pe.SolverFactory('gurobi') # Specify Solver
results = solver.solve(model, tee = False, keepfiles = False)
print("\n\nStatus:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# --------- POST-PROCESSING -------------------

states=[]
actions=[]
dual_vals=[]
for s in model.states:
    for a in model.actions:
        try:
            if (model.X[s,a].value != 0 and model.X[s,a].value != None) or (model.Y[s,a].value != 0 and model.Y[s,a].value != None):
                states=np.append(states,mdp.states[s])
                actions=np.append(actions,mdp.actions[a])
                dual_vals=np.append(dual_vals,np.round(model.X[s,a].value,10))
        except:
            print('Suspicious value at state %d action %d'%(s,a),model.X[s,a].value)
            continue
df=pd.DataFrame.from_dict(dict(state=states,action=actions,dual_val=dual_vals))
print("\n\nObjective function value: ", model.OBJ())
df[['state','action','dual_val']]



Status: ok
Termination Condition: optimal


Objective function value:  589.47899345471


,state,action,dual_val
0,No Improvement w/o AED,1st AED,0.000000
1,Poor control w/o AED,1st AED,0.000000
2,Improved but not controlled w/o AED,1st AED,0.000000
3,Refractory epilepsy,1st AED,0.129593
4,No Improvement with 1st AED,2nd AED,0.000000
5,Poor control with 1st AED,2nd AED,0.000000
6,Improved but not controlled with 1st AED,2nd AED,0.000000
7,Seizure free with drugs after 1st AED,Discontinue AED,0.000000
8,Relapse after SF when AED stopped,1st AED,0.000000
9,Poor control w/o AED after Relapse,1st AED,0.000000


# Surgery :Adverse Outcome at 22.48%

In [19]:
# %%time
InDir=r"C:\Users\Shreya\Dropbox\Shreya\Research Shreya\Epilepsy\Model"
filename='Data_Required_14b.xlsx'    
actions=['1st AED',
         '2nd AED',
         '3rd AED',
#          'Workup to Surgery',
         'Workup to Surgery with 3rd AED',
         'Surgery or resection',
         'Medical Management',
         'AED after SF',
         'Discontinue AED']   
# absorbing_states=['Refractory Epilepsy','Adverse Outcomes','']
qaly='readFile'
mdp=AvgCostMDP(InDir,filename,actions,qaly)

num_states=len(mdp.states)
num_actions=len(mdp.actions)

ai_s_RHS=[np.round(1.0/num_states,3)]*num_states
ai_s_RHS[0]=0.04#1-sum(ai_s_RHS[1:])

#------ Model Definition -------

# del model
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
model.states  = pe.Set(initialize = range(num_states))
# model.states  = pe.Set(initialize = range(num_states))
model.actions  = pe.Set(initialize = range(num_actions))
def cost_func(model,state,action): 
    return mdp.qalys[mdp.states[state]][mdp.actions[action]]
model.cost = pe.Param(model.states, model.actions, initialize = cost_func)
model.ai_s = pe.Param(model.states,   initialize = lambda model,s: ai_s_RHS[s]) #'s' is the index num of corresponding state
model.X = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)
model.Y = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)

#------------- CONSTRAINTS -----------
                         
def avg_cost_mdp_dual_constraint1(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_probab_transitions_with_X=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_X=sum_over_probab_transitions_with_X + pij*model.X[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X - sum_over_probab_transitions_with_X == 0)
model.Constraint1 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint1)
# model.Constraint1.pprint()

def avg_cost_mdp_dual_constraint2(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_actions_Y = sum(model.Y[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]]) 
    sum_over_probab_transitions_with_Y=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_Y=sum_over_probab_transitions_with_Y + pij*model.Y[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X + sum_over_actions_Y - sum_over_probab_transitions_with_Y == model.ai_s[j])
model.Constraint2 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint2)
# model.Constraint2.pprint()

#------ Objective Function ------

def obj_rule(model):
    return sum(model.cost[s,a] * model.X[s,a] for s in model.states for a in model.actions)
model.OBJ = pe.Objective(rule = obj_rule, sense = pe.maximize)
# model.OBJ.pprint()

#------ Solve LP ------

solver = pe.SolverFactory('gurobi') # Specify Solver
results = solver.solve(model, tee = False, keepfiles = False)
print("\n\nStatus:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# --------- POST-PROCESSING -------------------

states=[]
actions=[]
dual_vals=[]
for s in model.states:
    for a in model.actions:
        try:
            if (model.X[s,a].value != 0 and model.X[s,a].value != None) or (model.Y[s,a].value != 0 and model.Y[s,a].value != None):
                states=np.append(states,mdp.states[s])
                actions=np.append(actions,mdp.actions[a])
                dual_vals=np.append(dual_vals,np.round(model.X[s,a].value,10))
        except:
            print('Suspicious value at state %d action %d'%(s,a),model.X[s,a].value)
            continue
df=pd.DataFrame.from_dict(dict(state=states,action=actions,dual_val=dual_vals))
print("\n\nObjective function value: ", model.OBJ())
df[['state','action','dual_val']]



Status: ok
Termination Condition: optimal


Objective function value:  566.64122042317


,state,action,dual_val
0,No Improvement w/o AED,1st AED,0.000000
1,Poor control w/o AED,1st AED,0.000000
2,Improved but not controlled w/o AED,1st AED,0.000000
3,Refractory epilepsy,1st AED,0.100918
4,No Improvement with 1st AED,2nd AED,0.000000
5,Poor control with 1st AED,2nd AED,0.000000
6,Improved but not controlled with 1st AED,1st AED,0.000000
7,Seizure free with drugs after 1st AED,Discontinue AED,0.000000
8,Relapse after SF when AED stopped,1st AED,0.000000
9,Poor control w/o AED after Relapse,1st AED,0.000000


### Surgery Adverse 44.44% 

In [12]:
# %%time
InDir=r"C:\Users\Shreya\Dropbox\Shreya\Research Shreya\Epilepsy\Model"
filename='Data_Required_14c.xlsx'    
actions=['1st AED',
         '2nd AED',
         '3rd AED',
#          'Workup to Surgery',
         'Workup to Surgery with 3rd AED',
         'Surgery or resection',
         'Medical Management',
         'AED after SF',
         'Discontinue AED']   
# absorbing_states=['Refractory Epilepsy','Adverse Outcomes','']
qaly='readFile'
mdp=AvgCostMDP(InDir,filename,actions,qaly)

num_states=len(mdp.states)
num_actions=len(mdp.actions)

ai_s_RHS=[np.round(1.0/num_states,3)]*num_states
ai_s_RHS[0]=0.04#1-sum(ai_s_RHS[1:])

#------ Model Definition -------

# del model
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
model.states  = pe.Set(initialize = range(num_states))
# model.states  = pe.Set(initialize = range(num_states))
model.actions  = pe.Set(initialize = range(num_actions))
def cost_func(model,state,action): 
    return mdp.qalys[mdp.states[state]][mdp.actions[action]]
model.cost = pe.Param(model.states, model.actions, initialize = cost_func)
model.ai_s = pe.Param(model.states,   initialize = lambda model,s: ai_s_RHS[s]) #'s' is the index num of corresponding state
model.X = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)
model.Y = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)

#------------- CONSTRAINTS -----------
                         
def avg_cost_mdp_dual_constraint1(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_probab_transitions_with_X=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_X=sum_over_probab_transitions_with_X + pij*model.X[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X - sum_over_probab_transitions_with_X == 0)
model.Constraint1 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint1)
# model.Constraint1.pprint()

def avg_cost_mdp_dual_constraint2(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_actions_Y = sum(model.Y[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]]) 
    sum_over_probab_transitions_with_Y=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_Y=sum_over_probab_transitions_with_Y + pij*model.Y[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X + sum_over_actions_Y - sum_over_probab_transitions_with_Y == model.ai_s[j])
model.Constraint2 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint2)
# model.Constraint2.pprint()

#------ Objective Function ------

def obj_rule(model):
    return sum(model.cost[s,a] * model.X[s,a] for s in model.states for a in model.actions)
model.OBJ = pe.Objective(rule = obj_rule, sense = pe.maximize)
# model.OBJ.pprint()

#------ Solve LP ------

solver = pe.SolverFactory('gurobi') # Specify Solver
results = solver.solve(model, tee = False, keepfiles = False)
print("\n\nStatus:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# --------- POST-PROCESSING -------------------

states=[]
actions=[]
dual_vals=[]
for s in model.states:
    for a in model.actions:
        try:
            if (model.X[s,a].value != 0 and model.X[s,a].value != None) or (model.Y[s,a].value != 0 and model.Y[s,a].value != None):
                states=np.append(states,mdp.states[s])
                actions=np.append(actions,mdp.actions[a])
                dual_vals=np.append(dual_vals,np.round(model.X[s,a].value,10))
        except:
            print('Suspicious value at state %d action %d'%(s,a),model.X[s,a].value)
            continue
df=pd.DataFrame.from_dict(dict(state=states,action=actions,dual_val=dual_vals))
print("\n\nObjective function value: ", model.OBJ())
df[['state','action','dual_val']]



Status: ok
Termination Condition: optimal


Objective function value:  566.64122042317


,state,action,dual_val
0,No Improvement w/o AED,1st AED,0.000000
1,Poor control w/o AED,1st AED,0.000000
2,Improved but not controlled w/o AED,1st AED,0.000000
3,Refractory epilepsy,1st AED,0.100918
4,No Improvement with 1st AED,2nd AED,0.000000
5,Poor control with 1st AED,2nd AED,0.000000
6,Improved but not controlled with 1st AED,1st AED,0.000000
7,Seizure free with drugs after 1st AED,Discontinue AED,0.000000
8,Relapse after SF when AED stopped,1st AED,0.000000
9,Poor control w/o AED after Relapse,1st AED,0.000000


# QALYs close to each other result 

In [10]:
# %%time
InDir=r"C:\Users\Shreya\Dropbox\Shreya\Research Shreya\Epilepsy\Model"
filename='Data_Required_12_.xlsx'    
actions=['1st AED',
         '2nd AED',
         '3rd AED',
#          'Workup to Surgery',
         'Workup to Surgery with 3rd AED',
         'Surgery or resection',
         'Medical Management',
         'AED after SF',
         'Discontinue AED']   
# absorbing_states=['Refractory Epilepsy','Adverse Outcomes','']
qaly='readFile'
mdp=AvgCostMDP(InDir,filename,actions,qaly)

num_states=len(mdp.states)
num_actions=len(mdp.actions)

ai_s_RHS=[np.round(1.0/num_states,3)]*num_states
ai_s_RHS[0]=0.04#1-sum(ai_s_RHS[1:])

#------ Model Definition -------

# del model
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
model.states  = pe.Set(initialize = range(num_states))
# model.states  = pe.Set(initialize = range(num_states))
model.actions  = pe.Set(initialize = range(num_actions))
def cost_func(model,state,action): 
    return mdp.qalys[mdp.states[state]][mdp.actions[action]]
model.cost = pe.Param(model.states, model.actions, initialize = cost_func)
model.ai_s = pe.Param(model.states,   initialize = lambda model,s: ai_s_RHS[s]) #'s' is the index num of corresponding state
model.X = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)
model.Y = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)

#------------- CONSTRAINTS -----------
                         
def avg_cost_mdp_dual_constraint1(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_probab_transitions_with_X=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_X=sum_over_probab_transitions_with_X + pij*model.X[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X - sum_over_probab_transitions_with_X == 0)
model.Constraint1 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint1)
# model.Constraint1.pprint()

def avg_cost_mdp_dual_constraint2(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_actions_Y = sum(model.Y[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]]) 
    sum_over_probab_transitions_with_Y=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_Y=sum_over_probab_transitions_with_Y + pij*model.Y[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X + sum_over_actions_Y - sum_over_probab_transitions_with_Y == model.ai_s[j])
model.Constraint2 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint2)
# model.Constraint2.pprint()

#------ Objective Function ------

def obj_rule(model):
    return sum(model.cost[s,a] * model.X[s,a] for s in model.states for a in model.actions)
model.OBJ = pe.Objective(rule = obj_rule, sense = pe.maximize)
# model.OBJ.pprint()

#------ Solve LP ------

solver = pe.SolverFactory('gurobi') # Specify Solver
results = solver.solve(model, tee = False, keepfiles = False)
print("\n\nStatus:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# --------- POST-PROCESSING -------------------

states=[]
actions=[]
dual_vals=[]
for s in model.states:
    for a in model.actions:
        try:
            if (model.X[s,a].value != 0 and model.X[s,a].value != None) or (model.Y[s,a].value != 0 and model.Y[s,a].value != None):
                states=np.append(states,mdp.states[s])
                actions=np.append(actions,mdp.actions[a])
                dual_vals=np.append(dual_vals,np.round(model.X[s,a].value,10))
        except:
            print('Suspicious value at state %d action %d'%(s,a),model.X[s,a].value)
            continue
df=pd.DataFrame.from_dict(dict(state=states,action=actions,dual_val=dual_vals))
print("\n\nObjective function value: ", model.OBJ())
df[['state','action','dual_val']]



Status: ok
Termination Condition: optimal


Objective function value:  0.19556109815800005


,state,action,dual_val
0,No Improvement w/o AED,1st AED,0.000000
1,Poor control w/o AED,1st AED,0.000000
2,Improved but not controlled w/o AED,1st AED,0.000000
3,Refractory epilepsy,1st AED,0.100918
4,No Improvement with 1st AED,2nd AED,0.000000
5,Poor control with 1st AED,2nd AED,0.000000
6,Improved but not controlled with 1st AED,1st AED,0.000000
7,Seizure free with drugs after 1st AED,Discontinue AED,0.000000
8,Relapse after SF when AED stopped,1st AED,0.000000
9,Poor control w/o AED after Relapse,1st AED,0.000000


# QALY med = QALY surgery 

In [11]:
# %%time
InDir=r"C:\Users\Shreya\Dropbox\Shreya\Research Shreya\Epilepsy\Model"
filename='Data_Required_12_.xlsx'    
actions=['1st AED',
         '2nd AED',
         '3rd AED',
#          'Workup to Surgery',
         'Workup to Surgery with 3rd AED',
         'Surgery or resection',
         'Medical Management',
         'AED after SF',
         'Discontinue AED']   
# absorbing_states=['Refractory Epilepsy','Adverse Outcomes','']
qaly='readFile'
mdp=AvgCostMDP(InDir,filename,actions,qaly)

num_states=len(mdp.states)
num_actions=len(mdp.actions)

ai_s_RHS=[np.round(1.0/num_states,3)]*num_states
ai_s_RHS[0]=0.04#1-sum(ai_s_RHS[1:])

#------ Model Definition -------

# del model
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
model.states  = pe.Set(initialize = range(num_states))
# model.states  = pe.Set(initialize = range(num_states))
model.actions  = pe.Set(initialize = range(num_actions))
def cost_func(model,state,action): 
    return mdp.qalys[mdp.states[state]][mdp.actions[action]]
model.cost = pe.Param(model.states, model.actions, initialize = cost_func)
model.ai_s = pe.Param(model.states,   initialize = lambda model,s: ai_s_RHS[s]) #'s' is the index num of corresponding state
model.X = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)
model.Y = pe.Var(model.states, model.actions, domain = pe.NonNegativeReals)

#------------- CONSTRAINTS -----------
                         
def avg_cost_mdp_dual_constraint1(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_probab_transitions_with_X=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_X=sum_over_probab_transitions_with_X + pij*model.X[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X - sum_over_probab_transitions_with_X == 0)
model.Constraint1 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint1)
# model.Constraint1.pprint()

def avg_cost_mdp_dual_constraint2(model, j): #i is the state for which the constraint is being written
    sum_over_actions_X = sum(model.X[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]])
    sum_over_actions_Y = sum(model.Y[j,a] for a in model.actions if mdp.actions[a] in mdp.states_actions[mdp.states[j]]) 
    sum_over_probab_transitions_with_Y=0
    for i in model.states: #we will loop over all states j
        for a in model.actions:
            action=mdp.actions[a]
            if action in mdp.states_actions[mdp.states[i]]:
                transitions=mdp.transitions[action]
                if mdp.states[j] in transitions.columns:
                    try:
                        pij=transitions.at[mdp.states[i],mdp.states[j]]
                        if not(np.isnan(pij)):
                            sum_over_probab_transitions_with_Y=sum_over_probab_transitions_with_Y + pij*model.Y[i,a]  
                    except:
                        pij=0.0
    return(sum_over_actions_X + sum_over_actions_Y - sum_over_probab_transitions_with_Y == model.ai_s[j])
model.Constraint2 = pe.Constraint(model.states, rule = avg_cost_mdp_dual_constraint2)
# model.Constraint2.pprint()

#------ Objective Function ------

def obj_rule(model):
    return sum(model.cost[s,a] * model.X[s,a] for s in model.states for a in model.actions)
model.OBJ = pe.Objective(rule = obj_rule, sense = pe.maximize)
# model.OBJ.pprint()

#------ Solve LP ------

solver = pe.SolverFactory('gurobi') # Specify Solver
results = solver.solve(model, tee = False, keepfiles = False)
print("\n\nStatus:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# --------- POST-PROCESSING -------------------

states=[]
actions=[]
dual_vals=[]
for s in model.states:
    for a in model.actions:
        try:
            if (model.X[s,a].value != 0 and model.X[s,a].value != None) or (model.Y[s,a].value != 0 and model.Y[s,a].value != None):
                states=np.append(states,mdp.states[s])
                actions=np.append(actions,mdp.actions[a])
                dual_vals=np.append(dual_vals,np.round(model.X[s,a].value,10))
        except:
            print('Suspicious value at state %d action %d'%(s,a),model.X[s,a].value)
            continue
df=pd.DataFrame.from_dict(dict(state=states,action=actions,dual_val=dual_vals))
print("\n\nObjective function value: ", model.OBJ())
df[['state','action','dual_val']]



Status: ok
Termination Condition: optimal


Objective function value:  248.47725501450998


,state,action,dual_val
0,No Improvement w/o AED,1st AED,0.000000
1,Poor control w/o AED,1st AED,0.000000
2,Improved but not controlled w/o AED,1st AED,0.000000
3,Refractory epilepsy,1st AED,0.115520
4,No Improvement with 1st AED,2nd AED,0.000000
5,Poor control with 1st AED,2nd AED,0.000000
6,Improved but not controlled with 1st AED,1st AED,0.000000
7,Seizure free with drugs after 1st AED,Discontinue AED,0.000000
8,Relapse after SF when AED stopped,1st AED,0.000000
9,Poor control w/o AED after Relapse,1st AED,0.000000
